In [ ]:
import os
import sqlite3
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from rank_bm25 import BM25Okapi
import json 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from rank_bm25 import BM25Okapi

#Load env variables, pdf data
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
loader = PyPDFLoader("Panduan Lengkap Pengguna MikoVexa AI Update.pdf")
docs = loader.load()
full_text = "\n".join([page.page_content for page in docs])

#Use adaptive chunking for the chunking strategy (Section-Based Splitting)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=300)
pdf_chunks = text_splitter.split_text(full_text)

#Load SQL data and format it as JSON
def load_sql_data(db_path="mikovexa_synthetic_data.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()  
    tables = ["users", "mikovexa_robots", "commands", "security_logs"]
    sql_texts = []
    for table in tables:
        cursor.execute(f"PRAGMA table_info({table})")
        columns = [col[1] for col in cursor.fetchall()] 
        cursor.execute(f"SELECT * FROM {table} LIMIT 50")
        rows = cursor.fetchall()
        sql_texts.extend([json.dumps(dict(zip(columns, row))) for row in rows])
    conn.close()
    return sql_texts
sql_texts = load_sql_data()

#Create FAISS index from the synthetic data sources
all_texts = pdf_chunks + sql_texts  #Synthetic data source from PDF and SQL
embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5")
faiss_index_path = "mikovexa_faiss_index.bin"
if os.path.exists(faiss_index_path):
    faiss_index = faiss.read_index(faiss_index_path)
else:
    embeddings = embedding_model.encode(all_texts)
    faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
    faiss_index.add(np.array(embeddings))
    faiss.write_index(faiss_index, faiss_index_path)

#Create BM25 index
#Initialize Indonesian Stemmer for BM25
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def tokenize_indonesian(text):
    words = text.lower().split()
    return [stemmer.stem(word) for word in words]
bm25 = BM25Okapi([tokenize_indonesian(text) for text in all_texts if isinstance(text, str)])


#Structured SQL query handling
def query_sql_database(query):
    conn = sqlite3.connect("mikovexa_synthetic_data.db")
    cursor = conn.cursor()
    query_mappings = {
        "pengguna": "SELECT name, location FROM users LIMIT 3",
        "perintah": "SELECT command_text, timestamp FROM commands ORDER BY timestamp DESC LIMIT 3",
        "keamanan": "SELECT log_message, timestamp FROM security_logs ORDER BY timestamp DESC LIMIT 3"
    }
    for keyword, sql_query in query_mappings.items():
        if keyword in query.lower():
            cursor.execute(sql_query)
            results = cursor.fetchall()
            conn.close()
            return "\n".join([" | ".join(map(str, row)) for row in results])
    conn.close()
    return None

#Hybrid retrieval (FAISS + BM25 + SQL)
def retrieve_text(query, top_k=5):
    # Check SQL first
    sql_results = query_sql_database(query)
    if sql_results:
        return sql_results  # ✅ Prioritize SQL for structured data queries

    # If no SQL match, use FAISS + BM25
    query_embedding = embedding_model.encode([query])
    distances, indices = faiss_index.search(query_embedding, top_k)
    faiss_results = [all_texts[i] for i in indices[0]]

    # Multi-Hop Expansion
    related_indices = set()
    for idx in indices[0]:
        related_indices.update(range(max(0, idx - 1), min(len(all_texts), idx + 2)))
    related_chunks = [all_texts[i] for i in related_indices if i not in indices[0]]
    stemmed_query = tokenize_indonesian(query)
    bm25_results = bm25.get_top_n(stemmed_query, all_texts, n=top_k)

    return "\n".join(set(faiss_results + related_chunks + bm25_results))

#Query Groq API for the answer generation
llm = ChatGroq(model_name="llama3-8b-8192", api_key=groq_api_key)
def query_groq(query):
    retrieved_context = retrieve_text(query)[:1500]  

    #If nothing is found, we try with broader keyword-based BM25
    if not retrieved_context.strip():
        retrieved_context = retrieve_text(query + " terkait")[:1500]  

    #If nothing found, we try extracting SQL insights
    if not retrieved_context.strip():
        sql_context = query_sql_database(query)
        if sql_context:
            retrieved_context = sql_context  

    #Final Check(if nothing is found after multiple searches)
    if not retrieved_context.strip():
        retrieved_context = "Tidak ada informasi yang ditemukan dalam dokumen atau database MikoVexa."

    prompt = f"""
    Anda adalah asisten AI MikoVexa.
    Tugas Anda adalah memberikan jawaban akurat berdasarkan informasi yang tersedia di dokumen dan database.  
    - Jangan menebak jawaban.Jika informasi tidak tersedia, katakan dengan jelas bahwa jawaban tidak ditemukan.  
    - Gunakan format daftar jika terdapat langkah-langkah atau banyak detail.  
    - Jika jawaban tidak langsung tersedia, coba gunakan informasi terkait atau alternatif.
    - Jawab dalam bahasa Indonesia
    
    Konteks yang ditemukan: 
    {retrieved_context}

    Pertanyaan: {query}

    Jawaban:
    """
    response = llm.invoke(prompt)

    #Token Monitoring
    tokens_used = response.response_metadata.get('token_usage', {}).get('total_tokens', "Unknown")
    print(f"Tokens Used: {tokens_used} for query: {query}")

    return response

#10 questions for benchmarking
test_questions = [
    "Apakah MikoVexa menyimpan percakapan pengguna? Jika tidak, bagaimana sistem keamanannya bekerja?",
    "Bagaimana cara melihat riwayat interaksi pengguna dengan MikoVexa?",
    "Bagaimana cara memperbarui firmware MikoVexa secara otomatis dan manual?",
    "Bagaimana cara mengatur keamanan dan enkripsi data di MikoVexa?",
    "Bagaimana cara mengatur skenario otomatisasi perangkat pintar dengan MikoVexa?",
    "Bagaimana cara MikoVexa menangani upaya login yang gagal secara berulang?",
    "Bagaimana cara menghubungkan MikoVexa ke perangkat rumah pintar?",
    "Apa langkah-langkah untuk mengatasi kendala koneksi Wi-Fi pada MikoVexa?",
    "Bagaimana cara menggunakan fitur integrasi jadwal kerja di MikoVexa?",
    "Bagaimana cara pengguna mengelola akses multi-akun dalam aplikasi MikoVexa?",
]

for question in test_questions:
    answer = query_groq(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Tokens Used: 629 for query: Apakah MikoVexa menyimpan percakapan pengguna? Jika tidak, bagaimana sistem keamanannya bekerja?
Question: Apakah MikoVexa menyimpan percakapan pengguna? Jika tidak, bagaimana sistem keamanannya bekerja?
Answer: content='Berdasarkan informasi yang tersedia, tidak ada informasi yang spesifik tentang MikoVexa menyimpan percakapan pengguna. Namun, sebagai asisten AI, saya dapat memberikan informasi tentang sistem keamanan yang digunakan oleh MikoVexa.\n\nMikoVexa menggunakan teknologi keamanan yang efektif untuk melindungi data pengguna, termasuk informasi percakapan. Berikut adalah beberapa langkah yang diambil MikoVexa untuk melindungi data pengguna:\n\n1. **Enkripsi data**: MikoVexa menggunakan enkripsi data untuk melindungi informasi percakapan pengguna. Enkripsi data membuat informasi tidak dapat dibaca oleh pihak yang tidak berwenang.\n2. **Authentikasi**: MikoVexa menggunakan teknologi autentikasi untuk memastikan bahwa pengguna yang masuk ke sistem adal

In [9]:
import json
import fitz 

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_texts = [page.get_text("text") for page in doc]
    return all_texts

def format_for_finetuning(text_chunks, output_path, model_type):
    formatted_data = []
    
    for chunk in text_chunks:
        lines = chunk.split("\n")
        instruction = f"Jelaskan informasi berikut dalam bahasa sederhana:\n\n{lines[0]}" if lines else "Jelaskan informasi berikut:"
        response = " ".join(lines[1:]).strip() if len(lines) > 1 else "Maaf, tidak ada informasi yang tersedia."
        if model_type in ["llama", "gemma"]:
            entry = {
                "system": "Anda adalah asisten AI MikoVexa.",
                "instruction": instruction,
                "input": "",
                "output": response
            }
        elif model_type == "mistral":
            entry = {
                "prompt": instruction,
                "completion": response
            }
        elif model_type == "phi":
            entry = {
                "messages": [
                    {"role": "system", "content": "Anda adalah asisten AI MikoVexa."},
                    {"role": "user", "content": instruction},
                    {"role": "assistant", "content": response}
                ]
            }
        else:
            raise ValueError("Unsupported model type. Choose from: llama, gemma, mistral, phi.")
        
        formatted_data.append(entry)

    with open(output_path, "w", encoding="utf-8") as f:
        for entry in formatted_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Data formatted for {model_type} fine-tuning and saved to {output_path}")

pdf_path = "Panduan Lengkap Pengguna MikoVexa AI Update.pdf"
text_chunks = extract_text_from_pdf(pdf_path)

#Format the extracted texts for fine-tuning for different models
format_for_finetuning(text_chunks, "formatted_data_llama.jsonl", model_type="llama")
format_for_finetuning(text_chunks, "formatted_data_gemma.jsonl", model_type="gemma")
format_for_finetuning(text_chunks, "formatted_data_mistral.jsonl", model_type="mistral")
format_for_finetuning(text_chunks, "formatted_data_phi.jsonl", model_type="phi")

Data formatted for llama fine-tuning and saved to formatted_data_llama.jsonl
Data formatted for gemma fine-tuning and saved to formatted_data_gemma.jsonl
Data formatted for mistral fine-tuning and saved to formatted_data_mistral.jsonl
Data formatted for phi fine-tuning and saved to formatted_data_phi.jsonl
